# 一、高频分析

定义加载小说的函数

In [ ]:
def load_novel(novel):
    with open(f'./file/novels/{novel}.txt', 'rt', encoding='utf-8') as f:
        return f.read()

# 二、主角分析

加载人物数据

In [ ]:
with open('./file/data/names.txt', 'rt', encoding='utf-8') as f:
    data = [line.rstrip() for line in f]
    novels = data[::2]
    names = data[1::2]
    novel_names = {k: v.split() for k,v in zip(novels, names)}
    del novels, names, data

可以预览一下《天龙八部》中的人物

In [ ]:
",".join(novel_names['书剑恩仇录'][:20])

下面我们寻找一下每部小说的主角，统计每个人物的出场次数，显然次数越多主角光环越强，下面我们看看每部小说，出现次数最多的前十个人物：

In [ ]:
from collections import Counter
def find_main_characters(novel, num=10, content=None):
    if content is None:
        content = load_novel(novel)
    count = Counter()
    for name in novel_names[novel]:
        count[name] = content.count(name)
    return count.most_common(num)

for novel in novel_names:
    print(novel, find_main_characters(novel, num=5, content=None))
    

上述结果用文本展示了每部小说的前5个主角，但是不够直观，下面用pyecharts的树图展示一下：

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import TreeMap

data = []
for novel in novel_names:
    tmp = []
    data.append({"name": novel, "children": tmp})
    for name, count in find_main_characters(novel, 5):
        tmp.append({"name": name, "value": count})
c = (
    TreeMap()
    .add("", data, levels=[
        opts.TreeMapLevelsOpts(),
        opts.TreeMapLevelsOpts(
            color_saturation=[0.4, 0.6],
            treemap_itemstyle_opts=opts.TreeMapItemStyleOpts(
                border_color_saturation=0.7, gap_width=5, border_width=10
            ),
            upper_label_opts=opts.LabelOpts(
                is_show=True, position='insideTopLeft', vertical_align='top'
            )
        ),
    ])
    .set_global_opts(title_opts=opts.TitleOpts(title="金庸小说主角"))
)
c.render_notebook()

# 三、武功分析

In [ ]:
with open('./file/data/kungfu.txt', 'rt', encoding='utf-8') as f:
    data = [line.rstrip() for line in f]
    novels = data[::2]
    kungfu = data[1::2]
    novel_kungfu = {k: v.split() for k,v in zip(novels, kungfu) if k!= '未知'}
    del novels, kungfu, data
    
def find_main_kungfus(novel, num=10, content=None):
    if content is None:
        content = load_novel(novel)
    count = Counter()
    for kungfu in novel_kungfu[novel]:
        count[kungfu] = content.count(kungfu)
    return count.most_common(num)

data = []
for novel in novel_kungfu:
    tmp = []
    data.append({"name": novel, "children": tmp})
    for name, count in find_main_kungfus(novel, 5):
        tmp.append({"name": name, "value": count})
c = (
    TreeMap()
    .add("", data, levels=[
        opts.TreeMapLevelsOpts(),
        opts.TreeMapLevelsOpts(
            color_saturation=[0.4, 0.6],
            treemap_itemstyle_opts=opts.TreeMapItemStyleOpts(
                border_color_saturation=0.7, gap_width=5, border_width=10
            ),
            upper_label_opts=opts.LabelOpts(
                is_show=True, position='insideTopLeft', vertical_align='top'
            )
        ),
    ])
    .set_global_opts(title_opts=opts.TitleOpts(title="金庸小说主角"))
)
c.render_notebook()

# 四、门派分析

In [ ]:
with open('./file/data/bangs.txt', encoding="utf-8") as f:
    data = [line.rstrip() for line in f]
novels = data[::2]
bangs = data[1::2]
novel_bangs = {k: v.split() for k, v in zip(novels, bangs) if k != "未知"}
del novels, bangs, data


def find_main_bangs(novel, num=10, content=None):
    if content is None:
        content = load_novel(novel)
    count = Counter()
    for name in novel_bangs[novel]:
        count[name] = content.count(name)
    return count.most_common(num)

data = []
for novel in novel_bangs:
    tmp = []
    data.append({"name": novel, "children": tmp})
    for name, count in find_main_bangs(novel, 5):
        tmp.append({"name": name, "value": count})
c = (
    TreeMap()
    .add("", data, levels=[
        opts.TreeMapLevelsOpts(),
        opts.TreeMapLevelsOpts(
            color_saturation=[0.3, 0.6],
            treemap_itemstyle_opts=opts.TreeMapItemStyleOpts(
                border_color_saturation=0.7, gap_width=5, border_width=10
            ),
            upper_label_opts=opts.LabelOpts(
                is_show=True, position='insideTopLeft', vertical_align='top'
            )
        ),
    ])
    .set_global_opts(title_opts=opts.TitleOpts(title="金庸高频门派"))
)
c.render_notebook()

下面编写一个函数，输入一部小说名，可以输出其最高频的主角、武功和门派：

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Bar

def show_top10(novel):
    content = load_novel(novel)
    charecters = find_main_characters(novel, 10, content)[::-1]
    k, v = map(list, zip(*charecters))
    c = (
        Bar(init_opts=opts.InitOpts("720px", "320px"))
        .add_xaxis(k)
        .add_yaxis("", v)
        .reversal_axis()
        .set_series_opts(label_opts=opts.LabelOpts(position="right"))
        .set_global_opts(title_opts=opts.TitleOpts(title=f"{novel}主角"))
    )
    display(c.render_notebook())
    kungfus = find_main_kungfus(novel, 10, content)[::-1]
    k, v = map(list, zip(*kungfus))
    c = (
        Bar(init_opts=opts.InitOpts("720px", "320px"))
        .add_xaxis(k)
        .add_yaxis("", v)
        .reversal_axis()
        .set_series_opts(label_opts=opts.LabelOpts(position="right"))
        .set_global_opts(title_opts=opts.TitleOpts(title=f"{novel}功夫"))
    )
    display(c.render_notebook())
    bangs = find_main_bangs(novel, 10, content)[::-1]
    k, v = map(list, zip(*bangs))
    c = (
        Bar(init_opts=opts.InitOpts("720px", "320px"))
        .add_xaxis(k)
        .add_yaxis("", v)
        .reversal_axis()
        .set_series_opts(label_opts=opts.LabelOpts(position="right"))
        .set_global_opts(title_opts=opts.TitleOpts(title=f"{novel}门派"))
    )
    display(c.render_notebook())

In [ ]:
show_top10("天龙八部")

# 五、词云图分析

可以先添加所有的人物、武功和门派作为自定义词汇：

In [ ]:
import jieba

for novel, names in novel_names.items():
    for name in names:
        jieba.add_word(name)
        
for novel, kungfus in novel_kungfu.items():
    for kungfu in kungfus:
        jieba.add_word(kungfu)

for novel, bangs in novel_bangs.items():
    for bang in bangs:
        jieba.add_word(bang)

## 文章整体词云查看

这里我们仅提取词长度不小于4的成语、俗语和短语进行分析，以天龙八部这部小说为例：

In [ ]:
from IPython.display import Image
import stylecloud
import jieba
import re

text = re.sub("[^一-龟]+", " ", load_novel("天龙八部"))
words = [word for word in jieba.cut(text) if len(word) >=4]
stylecloud.gen_stylecloud(" ".join(words),
                          collocations=False,
                          font_path=r'C:\Windows\Fonts\msyhbd.ttc',
                          icon_name='fas fa-square',
                          output_name='tmp.png')
Image(filename='./file/tmp.png')

## 主角相关剧情词云

我们知道《神雕侠侣》这部小说最重要的主角是杨过和小龙女，我们可能会对于杨过和小龙女之间所发生的故事很感兴趣。如果通过程序快速了解呢？

我们考虑把《神雕侠侣》这部小说每一段中出现杨过及小龙女的段落进行jieba分词并制作词云。

同样我们只看4个字以上的词：

In [ ]:
data = []
for line in load_novel("神雕侠侣").splitlines():
    if "杨过" in line and "小龙女" in line:
        line = re.sub("[^一-龟]+", " ", line)
        data.extend(word for word in jieba.cut(line) if len(word) >= 4)
stylecloud.gen_stylecloud(" ".join(data),
                          collocations=False,
                          font_path=r'C:\Windows\Fonts\msyhbd.ttc',
                          icon_name='fas fa-square',
                          output_name='tmp.png')
Image(filename='./file/tmp.png')

# 六、人物关系分析

金庸小说15部小说中预计出现了1400个以上的角色，下面我们将遍历小说的每一段，在一段中出现的任意两个角色，都计数1。最终我们取出现频次最高的前200个关系对进行可视化。

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Graph
import math
import itertools

count = Counter()
for novel in novel_names:
    names = novel_names[novel]
    re_rule = f"({'|'.join(names)})"
    for line in load_novel(novel).splitlines():
        names = list(set(re.findall(re_rule, line)))
        if names and len(names) >= 2:
            names.sort()
            for s, t in itertools.combinations(names, 2):
                count[(s, t)] += 1
count = count.most_common(200)
node_count, nodes, links = Counter(), [], []
for (n1, n2), v in count:
    node_count[n1] += 1
    node_count[n2] += 1
    links.append({"source": n1, "target": n2})
for node, count in node_count.items():
    nodes.append({"name": node, "symbolSize": int(math.log(count)*5)+5})
c = (
    Graph(init_opts=opts.InitOpts("1280px","960px"))
    .add("", nodes, links, repulsion=30)
)
c.render("./file/tmp.html")

# 七、Word2Vec分析

Word2Vec是一款将词表征为实数值向量的高效工具，接下来，我们将使用它来处理这些小说。

首先我要将所有小说的段落分词后添加到组织到一起（前面的程序可以重启）：

In [ ]:
import jieba

def load_novel(novel):
    with open(f'./file/novels/{novel}.txt', 'rt', encoding='utf-8') as f:
        return f.read()
    
with open('./file/data/names.txt', 'rt', encoding='utf-8') as f:
    data = f.read().splitlines()
    novels = data[::2]
    names = []
    for line in data[1::2]:
        names.extend(line.split())
        
with open('./file/data/kungfu.txt', encoding="utf-8") as f:
    data = f.read().splitlines()
    kungfus = []
    for line in data[1::2]:
        kungfus.extend(line.split())

with open('./file/data/bangs.txt', encoding="utf-8") as f:
    data = f.read().splitlines()
    bangs = []
    for line in data[1::2]:
        bangs.extend(line.split())
        
for name in names:
    jieba.add_word(name)
for kungfu in kungfus:
    jieba.add_word(kungfu)
for bang in bangs:
    jieba.add_word(bang)
    
# 去重
names = list(set(names))
kungfus = list(set(kungfus))
bangs = list(set(bangs))

sentences = []
for novel in novels:
    print(f"处理：{novel}")
    for line in load_novel(novel).splitlines():
        sentences.append(jieba.lcut(line))

接下面我们使用Word2Vec训练模型：

In [ ]:
import gensim

model = gensim.models.Word2Vec(sentences)

模型训练耗时15秒，若训练耗时较长可以把训练好的模型存到本地：

In [ ]:
model.save("./file/louis_cha.model")

以后可以直接从本地磁盘读取模型：

In [ ]:
model = gensim.models.Word2Vec.load("./file/louis_cha.model")

有了模型，我们可以进行一些简单而有趣的测试。

**注意：每次生成的模型有一定随机性，后续结果根据生成的模型而变化，并非完全一致。**

## 相似角色、门派和武功

首先看与乔(萧)峰相似的角色：

In [ ]:
model.wv.most_similar(positive=["乔峰", "萧峰"])

除了角色，我们还可以看看门派：

In [ ]:
model.wv.most_similar(positive=["丐帮"])

还可以看看与降龙十八掌相似的武功秘籍：

In [ ]:
model.wv.most_similar(positive=["降龙十八掌"])

在 Word2Vec 的模型里，有过“中国-北京=法国-巴黎”的例子，我们看看"段誉"和"段公子"类似于乔峰和什么的关系呢？

In [ ]:
def find_relationship(a, b, c):
    d, _ = model.wv.most_similar(positive=[b, c], negative=[a])[0]
    print(f"{a}-{b} 犹如 {c}-{d}")


find_relationship("段誉", "段公子", "乔峰")

# 八、聚类分析

## 人物聚类分析

之前我们使用 Word2Vec 将每个词映射到了一个向量空间，因此，我们可以利用这个向量表示的空间，对这些词进行聚类分析。

首先取出所有角色对应的向量空间：

In [ ]:
import numpy as np

all_names = []
word_vectors = []
for name in names:            
    if name in model.wv:
        all_names.append(name)
        word_vectors.append(model.wv[name])
all_names = np.array(all_names)
word_vectors = np.vstack(word_vectors)

聚类算法有很多，这里我们使用基本的Kmeans算法进行聚类，如果只分成3类，那么很明显地可以将众人分成主角，配角，跑龙套的三类：

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd

N = 3
labels = KMeans(N).fit(word_vectors).labels_
df = pd.DataFrame({"name": all_names, "label": labels})
for label, names in df.groupby("label").name:
    print(f"类别{label}共{len(names)}个角色，前100个角色有：\n{','.join(names[:100])}\n")

我们可以根据每个类别的角色数量的相对大小，判断该类别的角色是属于主角，配角还是跑龙套。

下面我们过滤掉众龙套角色之后，重新聚合成四类：

In [ ]:
c = pd.Series(labels).mode().iat[0]
remain_names = all_names[labels != c]
remain_vectors = word_vectors[labels != c]
remain_label = KMeans(4).fit(remain_vectors).labels_
df = pd.DataFrame({"name": remain_names, "label": remain_label})
for label, names in df.groupby("label").name:
    print(f"类别{label}共{len(names)}个角色，前100个角色有：\n{','.join(names[:100])}\n")

每次运行结果都不一样，大家可以调整类别数量继续测试。从结果可以看到，反派更倾向于被聚合到一起，非正常姓名的人物更倾向于被聚合在一起，主角更倾向于被聚合在一起。

##  人物层级聚类

现在我们采用层级聚类的方式，查看人物间的层次关系，这里同样龙套角色不再参与聚类。

层级聚类调用 scipy.cluster.hierarchy 中层级聚类的包，在此之前先解决matplotlib中文乱码问题：

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

接下来调用代码为：

In [ ]:
import scipy.cluster.hierarchy as sch

y = sch.linkage(remain_vectors, method="ward")
_, ax = plt.subplots(figsize=(10, 80))
z = sch.dendrogram(y, orientation='right')
idx = z['leaves']
ax.set_xticks([])
ax.set_yticklabels(remain_names[idx], fontdict={'fontsize': 12})
ax.set_frame_on(False)
plt.show()

## 武功层级聚类

对各种武功作与人物层次聚类相同的操作：

In [ ]:
all_names = []
word_vectors = []
for name in kungfus:
    if name in model.wv:
        all_names.append(name)
        word_vectors.append(model.wv[name])
all_names = np.array(all_names)
word_vectors = np.vstack(word_vectors)

Y = sch.linkage(word_vectors, method="ward")

_, ax = plt.subplots(figsize=(10, 40))
Z = sch.dendrogram(Y, orientation='right')
idx = Z['leaves']
ax.set_xticks([])
ax.set_yticklabels(all_names[idx], fontdict={'fontsize': 12})
ax.set_frame_on(False)
plt.show()

## 门派层次聚类

最后我们对门派进行层次聚类：

In [ ]:
all_names = []
word_vectors = []
for name in bangs:
    if name in model.wv:
        all_names.append(name)
        word_vectors.append(model.wv[name])
all_names = np.array(all_names)
word_vectors = np.vstack(word_vectors)

Y = sch.linkage(word_vectors, method="ward")

_, ax = plt.subplots(figsize=(10, 25))
Z = sch.dendrogram(Y, orientation='right')
idx = Z['leaves']
ax.set_xticks([])
ax.set_yticklabels(all_names[idx], fontdict={'fontsize': 12})
ax.set_frame_on(False)
plt.show()